In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import random
import requests
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

# Load the intents data from the intents.json
with open('intents.json', 'r') as file:
    data = json.load(file)

# Initializing lemmatizer and creating a set of stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to preprocess the text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove punctuation and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum()]
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Creating a dictionary to hold the patterns and their corresponding tags
patterns_dict = {}
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns_dict[tuple(preprocess_text(pattern))] = intent['tag']

# Creating a dictionary to hold the responses and their corresponding tags
responses_dict = {}
for intent in data['intents']:
    responses_dict[intent['tag']] = intent['responses']

# Function to get a response from the chatbot
def get_response(user_input):
    user_input_tokens = preprocess_text(user_input)
    for pattern, tag in patterns_dict.items():
        if set(user_input_tokens).intersection(set(pattern)):
            return random.choice(responses_dict[tag])
    return get_gemini_response(user_input)

# Configure the Gemini API
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("No API key found. Please set the GOOGLE_API_KEY environment variable.")
genai.configure(api_key=api_key)

# Function to call Gemini API
def get_gemini_response(question):
    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(question)
    if response:
        return response.text
    else:
        return "I'm sorry, I don't understand."

# Chat loop
print("Chatbot: Hello! How can I help you today? (Type 'exit' to end)")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    response = get_response(user_input)
    print("Chatbot:", response)

import joblib
# Save the model using joblib
joblib.dump((patterns_dict, responses_dict), 'psychology_chatbot_model.joblib')


c:\Users\Nitish\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nitish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nitish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nitish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Chatbot: Hello! How can I help you today? (Type 'exit' to end)
Chatbot: Goodbye!


['psychology_chatbot_model.joblib']

In [2]:
%pip install -q -U google-generativeai


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

api_key = os.getenv('')
print(api_key)

None


In [4]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.
